####  Run this cell to set up and start your interactive session.


In [5]:
%idle_timeout 2880
%glue_version 5.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from pyspark.sql import Window
from pyspark.sql import functions as F
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

In [2]:
spark.conf.set("spark.sql.catalog.glue_catalog", "org.apache.iceberg.spark.SparkCatalog")
spark.conf.set("spark.sql.catalog.glue_catalog.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog")
spark.conf.set("spark.sql.catalog.glue_catalog.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
spark.conf.set("spark.sql.catalog.glue_catalog.warehouse", "s3://aws-ddf-pi-2025/processed/warehouse/")

In [7]:
df_filled_sample = spark.read.format("iceberg").load("glue_catalog.proyecto1db.stock_iceberg_sample")

In [6]:
df_filled_sample = (
    df_filled_sample
    .withColumn("pct_change_low_high", ((F.col("high") - F.col("low")) / F.col("open")) * 100)
    .withColumn("pct_change_open_close", ((F.col("close") - F.col("open")) / F.col("open")) * 100)
)

+------+---------+-----+----+------+------+-------+-----------+--------+-------------------+-------------------+---------------------+
|symbol|timestamp| open|high|   low| close| volume|trade_count|    vwap|    local_timestamp|pct_change_low_high|pct_change_open_close|
+------+---------+-----+----+------+------+-------+-----------+--------+-------------------+-------------------+---------------------+
|  ACTU|     NULL| 9.88|9.88|  8.17|9.2125|20649.0|      150.0|9.386387|2024-08-13 10:30:00| 17.307692307692317|    -6.75607287449393|
|  ACTU|     NULL|  9.0| 9.0|   7.5|   7.5|16030.0|       78.0|8.039615|2024-08-13 11:00:00| 16.666666666666664|  -16.666666666666664|
|  ACTU|     NULL|7.775| 8.4|   7.5|   8.4|10838.0|       57.0|8.057806|2024-08-13 11:30:00| 11.575562700964634|    8.038585209003216|
|  ACTU|     NULL|  8.4| 8.4|8.2165|  8.35| 6774.0|       22.0|8.288284|2024-08-13 12:00:00| 2.1845238095238146|  -0.5952380952381037|
|  ACTU|     NULL|8.625|9.18|  8.35|  8.35| 9178.0|    

In [7]:

w = Window.partitionBy("symbol").orderBy("local_timestamp")
df_filled_sample = df_filled_sample.withColumn(
    "pct_gap_open_prev_close",
    ((F.col("open") - F.lag("close").over(w)) / F.lag("close").over(w)) * 100
)

In [13]:
# Window: per symbol, ordered by timestamp
w_symbol_time = Window.partitionBy("symbol").orderBy("local_timestamp")

# Define lookback periods (in candles of 30 minutes)
periods = {
    "0d": 0 * 32,      # 0 candles
    "1d": 1 * 32,      # 32 candles
    "7d": 7 * 32,      # 224 candles
    "28d": 28 * 32,    # 896 candles
    "112d": 112 * 32,  # 3584 candles
    "365d": 365 * 32   # 11680 candles
}

# Add open prices from X periods ago
for label, shift in periods.items():
    df_filled_sample = df_filled_sample.withColumn(
        f"open_{label}_ago", F.lag("open", shift).over(w_symbol_time)
    ).withColumn(
        f"pct_change_{label}",
        ((F.col("close") - F.col(f"open_{label}_ago")) / F.col(f"open_{label}_ago")) * 100
    )

# Optionally, drop the intermediate open columns (to keep only pct_change columns)
cols_to_drop = [f"open_{label}_ago" for label in periods.keys()] + ["day_open"]
df_filled_sample = df_filled_sample.drop(*cols_to_drop)

In [21]:
df_filled_sample.select(
    "symbol",
    "local_timestamp",
    "open",
    "close",
    "pct_change_1d",
    "pct_change_7d"
).filter(
    (F.col("pct_change_1d").isNotNull()) & (F.col("symbol") == "ACTU")
).orderBy("local_timestamp").show()

+------+-------------------+-----+------+------------------+-------------+
|symbol|    local_timestamp| open| close|     pct_change_1d|pct_change_7d|
+------+-------------------+-----+------+------------------+-------------+
|  ACTU|2024-08-14 10:30:00|9.175|  9.05|-8.400809716599191|         NULL|
|  ACTU|2024-08-14 11:00:00|9.159| 9.145|1.6111111111111063|         NULL|
|  ACTU|2024-08-14 11:30:00| 9.18|   9.3|19.614147909967848|         NULL|
|  ACTU|2024-08-14 12:00:00| 9.41| 9.325|11.011904761904749|         NULL|
|  ACTU|2024-08-14 12:30:00| 9.35|9.1999| 6.665507246376806|         NULL|
|  ACTU|2024-08-14 13:00:00|  9.1|  9.31|11.497005988023963|         NULL|
|  ACTU|2024-08-14 13:30:00|9.275|   9.2| 7.886250366461442|         NULL|
|  ACTU|2024-08-14 14:00:00| 9.25|  9.18|7.9999999999999964|         NULL|
|  ACTU|2024-08-14 14:30:00| 9.24|9.3499| 9.998823529411762|         NULL|
|  ACTU|2024-08-14 15:00:00|  9.3|  9.25| 8.823529411764707|         NULL|
|  ACTU|2024-08-14 15:30:

In [16]:
df_filled_sample.cache()
df_filled_sample = df_filled_sample.orderBy(
    F.col("symbol").asc(),
    F.col("local_timestamp").asc()
)
df_filled_sample.show(200)

+------+---------+------+------+------+------+-------+-----------+--------+-------------------+-------------------+---------------------+-----------------------+--------------------+--------------------+-------------+--------------+---------------+---------------+
|symbol|timestamp|  open|  high|   low| close| volume|trade_count|    vwap|    local_timestamp|pct_change_low_high|pct_change_open_close|pct_gap_open_prev_close|       pct_change_0d|       pct_change_1d|pct_change_7d|pct_change_28d|pct_change_112d|pct_change_365d|
+------+---------+------+------+------+------+-------+-----------+--------+-------------------+-------------------+---------------------+-----------------------+--------------------+--------------------+-------------+--------------+---------------+---------------+
|  ACTU|     NULL|  9.88|  9.88|  8.17|9.2125|20649.0|      150.0|9.386387|2024-08-13 10:30:00| 17.307692307692317|    -6.75607287449393|                   NULL|   -6.75607287449393|                NULL|  